In [1]:
import requests
import pandas as pd
from pprint import pprint
from operator import itemgetter
import json
import time
from datetime import datetime

from functools import reduce

from bs4 import BeautifulSoup

import random

# Сбор данных о пользователях, поставихших лайк посту

In [2]:
url = "https://www.instagram.com/p/BW1154FAhJz/"

In [251]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

# настройки для запуска в фоновом режиме
opts = Options()
opts.set_headless()
assert opts.headless

d:\coding\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use setter for headless property instead of set_headless
  


In [ ]:
# Подготовка словаря с именами html атрибутов для поиска элементов на странице
# with open("patterns.json", "w") as f:
#     json.dump({"liked_by_user": class_pattern, "liked_by_button": 'zV_Nj'}, f)

# Скроллинг

In [295]:
with open("patterns.json") as f:
    patterns = json.load(f)
    
url = "https://www.instagram.com/p/"

def get_likes(post_id):
    
#     print(f"Collecting likes from {post_id}")
    browser.get(f"{url}{post_id}")
    
    time.sleep(2)  # waiting for post page to load
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # scroll down to avoid Sign in message
    browser.find_element_by_xpath("//a[@class='{}']".format(patterns['liked_by_button'])).click()

    time.sleep(1)  # waiting for /liked_by/ page to load
    frame = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div")
    last_height = browser.execute_script("return arguments[0].scrollHeight", frame)
    users = set()
    
#     time.sleep(1)  # waiting for smth >.<
    while True:
        users = users.union({name.strip().split("\n")[0] for name in frame.text.split("\nПодписаться")[:-1]})  # -1 is usually ""
        browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", frame)  # рабочий вариант

        new_height = browser.execute_script("return arguments[0].scrollHeight", frame)
#         print(last_height, new_height)
        if new_height == last_height:
            break
        last_height = new_height
        time.sleep(0.5)
    print(f"Collected {len(users)} likes from {post_id}")
    return users

In [162]:
main_data = pd.read_csv("./data/main_data.csv", sep=";")
print(main_data.shape)
main_data.head()

(27562, 10)


,post_id,text,date,likes,owner_id,is_video,by_tag,phone_number,direct,price
0,3-9jrIxI65,"@damarkelova ・・・\r\nДоброе утро, друзья!! Отхв...",1.434444e+09,199,1.101804e+09,False,акварельмк,89037727778,NaN,2200
1,3vos8XRI98,"Вечером в среду, после обеда...\r\nЖдем вас на...",1.433930e+09,152,1.101804e+09,False,акварельмк,89037727778,NaN,2200
2,3vxZIas4-H,#Repost @culture_club with @repostapp.\r\n・・・\...,1.433935e+09,36,1.132830e+07,False,акварельмк,89037727778,NaN,2200
3,6ey5mMh57E,"Как и обещали, с радостью представляем вам нов...",1.439807e+09,9,9.591155e+08,False,акварельмк,8-963-328-37-28,NaN,2000
4,BhRt7-NF6lS,"💞 Дорогие подписчики 💞\r\n.\r\n🌸 21 апреля, су...",1.523120e+09,50,1.909118e+09,False,акварельмк,7951-677-6710,NaN,2500


In [294]:
browser = Firefox(options=opts)

In [ ]:
browser.get("https://www.instagram.com/p/BW1154FAhJz/")

In [291]:
likes = pd.DataFrame()

In [289]:
main_data[main_data["post_id"] == "3vos8XRI98"]["likes"].iloc[0]

152

In [301]:
main_data[~main_data["is_video"]]["post_id"]

0         3-9jrIxI65
1         3vos8XRI98
2         3vxZIas4-H
3         6ey5mMh57E
4        BhRt7-NF6lS
5        BmGuAi3nd7Y
6        BpLsN7OHZY1
7        BsC3i2Ijnha
8         zg74v5xI_B
9        BS21buoAVcd
10       BTPtxhfg0TE
11       BV2GCKGl7Yi
12       BV3oZB0F48m
13       BWOujDtFcRm
14       BWhi1SDlTlD
15       BanwyeUl1eJ
16       Bdn72uKFM0F
17       Bg4O7nllQmb
18       Bg8kyvQlVKd
20       BoSCAoaFZKx
21       BoboTSmlwjG
22       BpM5G-0gr9I
23       BeS4pueljmY
25       BefsgNwHUQN
27       BkS4B8JHJSN
29       BmcmS7mgaKc
30       BmdWaf8gaWQ
31       BmflzLkBdnx
33       BrNXio2FmGf
34       BC_LI0LqklO
            ...     
27531    BRvXwnfAknc
27532    BS5d_k1lgei
27533    BUH_zrihwBB
27534    BV12G33hVug
27535    BVy0rsSHncv
27536    BW8JzukHH4f
27537    BWXgGVvHpRF
27538    BXA0C_8Hnu6
27539    BXGF4ZynEKM
27540    BYDtEPMnzrj
27541    BYQ5w52npcB
27542    BZJWr8shr4j
27543    BaBYRAJBM2v
27545    Bc9iowYnQmr
27546    BcWYScUBmnn
27547    BdhjSVEBubs
27548    Bf45

# TODO
Разобраться с настойками ожидания - в определенный момент увеличивается время, необходиое на загрузку страницы после клика на кнопку

In [304]:
retry = 5
retry_delay = 5
for index, post in main_data[~main_data["is_video"]]["post_id"].loc[401:1000].iteritems():
    likes_in_data = main_data[main_data["post_id"] == post]["likes"].iloc[0]
    print(f"On index {index}")
    got_data = False
    for i in range(retry):
        try:
            user_set = get_likes(post)
            if len(user_set) == 0 and likes_in_data != 0:  # fix: it just takes much time sometimes to load the page!
#                 print(f"Got zero likes from post {post}")
                time.sleep(retry_delay * i**2)  # fix to exponential delay time growth
                browser.refresh()
                continue
            got_data = True
            break
        except NoSuchElementException as e:  # same as 200 response
            print(e)
            time.sleep(retry_delay * i**2)
    
    if not got_data:
        print(f"Unable to get likes from {post}")
        
    post_df = pd.DataFrame({user_name: 1 for user_name in user_set}, index=[post])
    likes = pd.concat([likes, post_df], sort=False)
    print(f"Likes shape is {likes.shape}")

On index 401
Collected 18 likes from BgjI6AUFVGI
Likes shape is (461, 17148)
On index 402
Collected 0 likes from BgrHZxplaWr
Collected 0 likes from BgrHZxplaWr
Collected 22 likes from BgrHZxplaWr
Likes shape is (462, 17148)
On index 403
Collected 67 likes from Bgv2ZnKB9vp
Likes shape is (463, 17148)
On index 405
Collected 63 likes from Bh1f1ymHrxx
Likes shape is (464, 17152)
On index 406
Collected 57 likes from Bh6yDunHj9Z
Likes shape is (465, 17158)
On index 407
Collected 18 likes from Bh7Qh_yAdml
Likes shape is (466, 17158)
On index 408
Collected 0 likes from Bh9rMPRAd8J
Collected 0 likes from Bh9rMPRAd8J
Collected 42 likes from Bh9rMPRAd8J
Likes shape is (467, 17169)
On index 409
Collected 76 likes from BhI1yrXBbbi
Likes shape is (468, 17176)
On index 411
Collected 38 likes from Bh_aKFNgTBv
Likes shape is (469, 17190)
On index 412
Collected 15 likes from Bhbbk6rFdIQ
Likes shape is (470, 17190)
On index 413
Collected 20 likes from BhbzlvBFoBz
Likes shape is (471, 17194)
On index 414


Likes shape is (522, 17638)
On index 478
Collected 48 likes from BllB6j5lHF4
Likes shape is (523, 17683)
On index 479
Collected 11 likes from BllaHMBBzOO
Likes shape is (524, 17689)
On index 481
Collected 11 likes from Bm2npBbBsiC
Likes shape is (525, 17693)
On index 483
Collected 11 likes from BmQXlxMlIKy
Likes shape is (526, 17704)
On index 484
Collected 11 likes from BmQa3ZVFKZr
Likes shape is (527, 17715)
On index 486
Collected 98 likes from BmfqeiVnrjI
Likes shape is (528, 17741)
On index 487
Collected 66 likes from Bn1jcfbHE57
Likes shape is (529, 17761)
On index 488
Collected 22 likes from Bn8WYGrBO14
Likes shape is (530, 17783)
On index 489
Collected 27 likes from Bn8ki1Sh4LY
Likes shape is (531, 17806)
On index 491
Collected 11 likes from BnROzYwHYYP
Likes shape is (532, 17811)
On index 492
Collected 76 likes from BnZMRyQn29F
Likes shape is (533, 17823)
On index 493
Collected 96 likes from Bn_XfrYHEmo
Likes shape is (534, 17843)
On index 494
Collected 113 likes from BnbskBuHs2

Collected 0 likes from BpUaf3DnQZp
Collected 0 likes from BpUaf3DnQZp
Collected 25 likes from BpUaf3DnQZp
Likes shape is (573, 19184)
On index 537
Collected 152 likes from BpbaBJMhGto
Likes shape is (574, 19335)
On index 538
Collected 75 likes from BpbeFQBAtir
Likes shape is (575, 19344)
On index 539
Collected 11 likes from Bpe994jAFl3
Likes shape is (576, 19347)
On index 540
Collected 145 likes from Bpg4PGwBSiJ
Likes shape is (577, 19440)
On index 541
Collected 18 likes from BpjXi-iAorz
Likes shape is (578, 19442)
On index 542
Collected 48 likes from BptRd_mB9gO
Likes shape is (579, 19484)
On index 543
Collected 107 likes from Bpuf3ePg-sC
Likes shape is (580, 19503)
On index 544
Collected 132 likes from Bpxmvf5DOuR
Likes shape is (581, 19523)
On index 545
Collected 11 likes from BqFDTSwg9q6
Likes shape is (582, 19526)
On index 546
Collected 11 likes from BqH_nNOjEkx
Likes shape is (583, 19531)
On index 547
Collected 11 likes from BqKmh1dAyBQ
Likes shape is (584, 19535)
On index 548
Co

WebDriverException: Message: Reached error page: about:neterror?e=dnsNotFound&u=https%3A//www.instagram.com/p/BsNM0gKATAP/liked_by/&c=UTF-8&f=regular&d=%D0%9C%D1%8B%20%D0%BD%D0%B5%20%D0%BC%D0%BE%D0%B6%D0%B5%D0%BC%20%D0%BF%D0%BE%D0%B4%D0%BA%D0%BB%D1%8E%D1%87%D0%B8%D1%82%D1%8C%D1%81%D1%8F%20%D0%BA%20%D1%81%D0%B5%D1%80%D0%B2%D0%B5%D1%80%D1%83%20www.instagram.com.


In [250]:
likes.sum().sort_values(ascending=False)

edinoroshka_spb            12.0
poisk_modeli_msk           11.0
ivanych67ai                11.0
keratin_botox_hair_spb     11.0
anna___orange              11.0
karina_bohon               11.0
zusva                      10.0
acwedding_official         10.0
annadyoma                  10.0
ankirju                    10.0
belochka_clayart           10.0
volkov.dmitrii.ev           9.0
poisk_modeli_spb_           9.0
chernova_nina555            9.0
ulyna_kor                   8.0
gubskaia.nadezhda           8.0
iriwka_n                    8.0
dasha._.piskareva           7.0
elena_al.va                 7.0
babochka_studios            7.0
solovushka_jewelry          6.0
irinashi                    6.0
divauliy                    5.0
nik_kiriukhin               5.0
mishki.zaiki                5.0
_katerina_khokhlova_        5.0
elenamuravenko_lampwork     5.0
galina.19800                5.0
_anasteziya_                5.0
julieenot                   5.0
                           ... 
larysa_r

In [305]:
likes.to_csv("./data/likes.csv", sep=";")

In [111]:
browser.execute_script("return arguments[0].scrollHeight", frame)

2240

Куски со stackoverflow

In [ ]:
data = [i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))]
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    data.extend([i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))])
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height